<a href="https://colab.research.google.com/github/Knightler/Kaggle-Competitions/blob/main/Open_Data_Day_2025_Dates_Types_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, random_split
from torch.optim.swa_utils import AveragedModel, SWALR, update_bn
from PIL import Image
import kagglehub

In [2]:
# --------------------------
# Setup device and seeds
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [ ]:
# --------------------------
# Data Preparation
# --------------------------
data_dir = kagglehub.dataset_download("wadhasnalhamdan/date-fruit-image-dataset-in-controlled-environment")
print("Path to dataset files:", data_dir)

 92%|█████████▏| 2.86G/3.11G [01:38<00:08, 30.6MB/s]

In [ ]:
desired_classes = ['Ajwa', 'Nabtat Ali', 'Sokari']

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=10, shear=5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
val_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)

In [ ]:
desired_idxs = [i for i, (_, label) in enumerate(full_dataset)
                if full_dataset.classes[label] in desired_classes]
subset_dataset = Subset(full_dataset, desired_idxs)

In [ ]:
train_size = int(0.8 * len(subset_dataset))
val_size = len(subset_dataset) - train_size
train_dataset, val_dataset = random_split(subset_dataset, [train_size, val_size])

val_dataset.dataset.transform = val_transforms

In [ ]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=4, pin_memory=True)

In [ ]:
# --------------------------
# Model Definition: EfficientNet-B0
# --------------------------
from torchvision.models import efficientnet_b0
model = efficientnet_b0(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(desired_classes))

In [ ]:
for param in model.parameters():
    param.requires_grad = True
model = model.to(device)

In [ ]:
# --------------------------
# Loss, Optimizer, and Scheduler
# --------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

In [ ]:
# --------------------------
# SWA Setup
# --------------------------
swa_model = AveragedModel(model)
swa_scheduler = SWALR(optimizer, swa_lr=0.01, anneal_strategy="cos", anneal_epochs=5)

In [ ]:
# --------------------------
# MixUp Function
# --------------------------
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [ ]:
# --------------------------
# Training Loop with Early Stopping, MixUp, and SWA
# --------------------------
num_epochs = 30
best_val_acc = 0.0
early_stopping_counter = 0
patience = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        # Apply MixUp randomly 50% of the time
        if random.random() > 0.5:
            inputs_mixed, labels_a, labels_b, lam = mixup_data(inputs, labels, alpha=1.0)
            outputs = model(inputs_mixed)
            loss = lam * criterion(outputs, labels_a) + (1 - lam) * criterion(outputs, labels_b)
        else:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += (outputs.argmax(dim=1) == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset)

    # Validation Phase
    model.eval()
    val_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            val_corrects += (outputs.argmax(dim=1) == labels).sum().item()
    val_acc = val_corrects / len(val_dataset)

    # Step the scheduler
    scheduler.step(val_acc)

    # Early Stopping Check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    # Apply SWA during the last 5 epochs
    if epoch >= (num_epochs - 5):
        swa_model.update_parameters(model)
        swa_scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.4f} - Val Acc: {val_acc:.4f}")

    if early_stopping_counter >= patience:
        print("Early stopping triggered!")
        break

# Finalize SWA
update_bn(train_loader, swa_model)
torch.save(swa_model.state_dict(), "swa_best_model.pth")
print("Training complete. Best model saved as 'best_model.pth' and SWA model as 'swa_best_model.pth'.")

In [ ]:
# --------------------------
# Test-Time Augmentation (TTA) for Prediction
# --------------------------
def predict_tta(image_path, model, transforms, class_names, num_augmentations=5):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    predictions = []
    for _ in range(num_augmentations):
        img = transforms(image).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(img)
            _, pred = torch.max(outputs, 1)
            predictions.append(pred.item())
    final_prediction = max(set(predictions), key=predictions.count)
    return class_names[final_prediction]

In [ ]:
class_to_idx = {cls: i for i, cls in enumerate(desired_classes)}
idx_to_class = {v: k for k, v in class_to_idx.items()}